### Numpy
在介绍PyTorch之前，我们将首先使用numpy实现网络。

Numpy提供了一个n维数组对象，以及许多用于操作这些数组的函数。Numpy是科学计算的通用框架; 它对计算图，深度学习或渐变都一无所知。但是，通过使用numpy操作手动实现网络的前向和后向传递，我们可以轻松地使用numpy将两层网络适配到随机数据：

In [1]:
import numpy as np
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
# H是隐藏层维度不是层个数，也就是那一层神经元个数
N, D_in, H, D_out = 64, 1000, 100,10
# D_in和D_out是每个数据对应的维度,所以一批数据的维度就是 （N，D）

# 创建随机输入与输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in,H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-7
for t in range(200):
    # 前向传播：计算预测值y,
    h = x.dot(w1)
    h_relu = np.maximum(h,0)
    y_pred = h_relu.dot(w2)
    
    # 计算损失函数
    loss = np.square(y_pred - y).sum()
    print(t,loss)
    
    # 反向传播，由链式法则计算w1 w2 对应loss的梯度
    grad_y_pred = 2*(y_pred - y)  # loss = sum[(y_pred - y)^2]
    grad_w2 = h_relu.T.dot(grad_y_pred) # y_pred = h_relu.dot(w2)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h<0] = 0 # relu函数小于0部分梯度为0
    grad_w1 = x.T.dot(grad_h)
    
    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
        

0 24208137.623141058
1 21850789.227255687
2 20132483.659388423
3 18809718.253678717
4 17739305.687064625
5 16835282.08636658
6 16044345.75576635
7 15333861.612287212
8 14684277.700018479
9 14082873.473491767
10 13520819.090716593
11 12992805.307806227
12 12494724.894214015
13 12023931.93026122
14 11577774.984947763
15 11153811.397413373
16 10750404.057258168
17 10366267.669776822
18 10000518.153287828
19 9651780.368189756
20 9319203.428830683
21 9001938.173719611
22 8698757.798353303
23 8408929.687612012
24 8131594.525932021
25 7866145.212545418
26 7611917.525715859
27 7368258.793255724
28 7134619.624340925
29 6910538.356527424
30 6695460.0783165125
31 6488975.710196834
32 6290684.549237605
33 6100250.016120296
34 5917180.034794411
35 5741172.722602271
36 5571907.123673895
37 5409181.181129552
38 5252615.916038133
39 5101879.317655962
40 4956848.341027949
41 4817117.203803131
42 4682424.337819265
43 4552552.354583254
44 4427290.216485217
45 4306608.920590263
46 4190167.2602110244
47 40

### pytorch: tensors  
Numpy是一个很棒的框架，但它不能利用GPU来加速其数值计算。对于现代深度神经网络，GPU通常提供50倍或更高的加速，因此不幸的是，numpy对于现代深度学习来说还不够。

在这里，我们介绍最基本的PyTorch概念：Tensor。PyTorch Tensor在概念上与numpy数组相同：Tensor是一个n维数组，PyTorch提供了许多用于在这些Tensors上运算的函数。在幕后，Tensors可以跟踪计算图和渐变，但它们也可用作科学计算的通用工具。

与numpy不同，PyTorch Tensors可以利用GPU加速其数值计算。要在GPU上运行PyTorch Tensor，只需将其转换为新的数据类型即可。

在这里，我们使用PyTorch Tensors将双层网络与随机数据相匹配。就像上面的numpy示例一样，我们需要手动实现网络中的前向和后向传递：

In [1]:
import torch
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # forward prop
    h = x.mm(w1)
    h_relu = h.clamp(min=0) # clamp：夹紧，所以意思你懂的
    y_pred = h_relu.mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # backward prop
    grad_y_pred = 2 * (y_pred-y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t()) # .t() 等于 numpy 中的.T,及转置
    grad_h = grad_h_relu.clone()# .clone 等于numpy 中的.copy()
    grad_h[h<0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    

0 28769096.0
1 26723762.0
2 29707274.0
3 32923014.0
4 31857936.0
5 24735096.0
6 15197170.0
7 7852862.0
8 3865950.0
9 2053918.25
10 1259704.75
11 888136.25
12 689210.375
13 565040.25
14 477093.375
15 409459.9375
16 354895.3125
17 309766.75
18 271899.375
19 239773.03125
20 212273.703125
21 188583.84375
22 168083.84375
23 150292.96875
24 134726.46875
25 121079.5078125
26 109071.765625
27 98466.65625
28 89085.84375
29 80753.0234375
30 73328.59375
31 66704.0
32 60783.375
33 55472.79296875
34 50699.71875
35 46411.6875
36 42543.03125
37 39042.91796875
38 35870.75390625
39 32992.2890625
40 30374.98828125
41 27994.421875
42 25823.0546875
43 23840.9609375
44 22029.70703125
45 20372.41796875
46 18853.55859375
47 17460.16015625
48 16180.5947265625
49 15004.806640625
50 13923.0615234375
51 12926.84765625
52 12009.02734375
53 11162.423828125
54 10381.048828125
55 9659.2939453125
56 8991.951171875
57 8374.7509765625
58 7803.63525390625
59 7274.30322265625
60 6783.6318359375
61 6328.546875
62 5906.463

### PyTorch：Tensors和autograd
在上面的例子中，我们不得不手动实现神经网络的前向和后向传递。手动实现反向传递对于小型双层网络来说并不是什么大问题，但对于大型复杂网络来说很快就会变得非常繁琐。

值得庆幸的是，我们可以使用自动微分 来自动计算神经网络中的后向传递。PyTorch中的 autograd包提供了这个功能。使用autograd时，网络的正向传递将定义 计算图形 ; 图中的节点将是张量，边将是从输入张量产生输出张量的函数。通过此图反向传播，您可以轻松计算渐变。

这听起来很复杂，在实践中使用起来非常简单。每个Tensor表示计算图中的节点。如果x是Tensor x.requires_grad=True那么x.grad是另一个Tensor持有x相对于某个标量值的梯度。

在这里，我们使用PyTorch Tensors和autograd来实现我们的双层网络; 现在我们不再需要手动实现通过网络的反向传递：

In [3]:
import time
import torch

In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
t1 = time.time()
N, D_in, H, D_out = 64, 1000, 100, 10

# 设定 requires_grad=False 表明我们在反向传播中不需要计算相应的张量的梯度
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 设定 requires_grad=True 表明我们在反向传播中会计算相应的张量的梯度
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # forword prop
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # loss function
    loss = (y_pred - y).pow(2).sum()
#     print(t, loss.item())
    
    # 利用autograd去计算loss函数的反向传播中所有梯度
    loss.backward()
    
    # 更新权重
    # 这里选择利用梯度下降手动更新权重，当然也可以使用torch.optim.SGD进行梯度下降
    with torch.no_grad():
        w1 -= learning_rate * w1.grad #因为前面的.backward,所以这里w1.grad直接获取到w1的梯度
        w2 -= learning_rate * w2.grad
        
        # 手动将梯度归零,否则梯度会累加，这是pytorch动态图的特点之一
        w1.grad.zero_()
        w2.grad.zero_()
        
t2 = time.time()
print(loss)
print(t2-t1,'s')

tensor(1.1734e-06, device='cuda:0', grad_fn=<SumBackward0>)
2.735474109649658 s


### PyTorch：定义新的autograd函数
每个原始 autograd 运算符实际上是两个在Tensors上运行的函数.  
